<a href="https://colab.research.google.com/github/rajbircit/assignments/blob/main/dl_a1/Question1_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Show the environment

In [1]:
!nvidia-smi

Sat Apr 17 09:40:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setup logging

In [2]:
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import logging
import sys
import random
import matplotlib.pyplot as plt

np.random.seed(1)
random.seed(1)
tf.random.set_seed(1)

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

# Open and read the Fashion MNIST database
Code copied from sample provided

In [3]:

def get_fashion_mnist_data()->tuple:
  """
  Returns the training (X,Y) and evaluation (X,Y)
  The features have already been reshaped and converted to
  floats in the range [0,1]

  Also, the Y has been converted to categorical data.
  """
  # load the training and test data    
  fashion_mnist = tf.keras.datasets.fashion_mnist
  (tr_x, tr_y), (te_x, te_y) = fashion_mnist.load_data()
  
  # reshape the feature data
  tr_x = tr_x.reshape(tr_x.shape[0], 784)
  te_x = te_x.reshape(te_x.shape[0], 784)
  
  # noramlise feature data
  tr_x = tr_x / 255.0
  te_x = te_x / 255.0
  logging.debug( "Shape of training features " + str(tr_x.shape))
  logging.debug( "Shape of test features " + str(te_x.shape))
  
  # one hot encode the training labels and get the transpose
  tr_y = np_utils.to_categorical(tr_y,10)
  tr_y = tr_y.T
  logging.debug("Shape of training labels " + str(tr_y.shape))
  
  # one hot encode the test labels and get the transpose
  te_y = np_utils.to_categorical(te_y,10)
  te_y = te_y.T
  logging.debug("Shape of testing labels " + str(te_y.shape))

  tr_x = tf.convert_to_tensor(tr_x, dtype=tf.float32)
  tr_y = tf.convert_to_tensor(tr_y, dtype=tf.float32)
  te_x = tf.convert_to_tensor(te_x, dtype=tf.float32)
  te_y = tf.convert_to_tensor(te_y, dtype=tf.float32)

  return tr_x, tr_y, te_x, te_y

# Some utility functions
1. Forward Pass
2. Cross Entropy
3. Accuracy Score
4. Flatten Network
5. Optimize Network (Adam)

In [4]:
def make_layer(shape:list, layername:str)->tf.Variable:
  return tf.Variable(tf.random.normal(shape, mean=0, stddev=0.05, \
                                      dtype=tf.dtypes.float32),
                                      name=layername)

# This function implements the softmax function
def softmax(a:tf.Tensor, axis:int=None)->tf.Tensor:
  t = tf.math.exp(a)
  s = tf.reduce_sum(t, axis=0)
  return tf.math.divide(t, s)

# This function implements the cross-entropy loss
def cross_entropy(y_true, y_pred)->tf.Tensor:
  # First clip the values to 1.0 x 10^(-8) because if there
  # are any values which are 0, then this will cause an error
  y_pred = tf.clip_by_value(y_pred, \
                            clip_value_min=1.0e-10, \
                            clip_value_max=tf.reduce_max(y_pred))
  y_pred = tf.math.multiply(y_true, tf.math.log(y_pred))
  y_pred = tf.multiply(y_pred, -1)
  # Find the average cross entropy, divide by number of samples
  divisor = y_pred.shape[1] if len(y_pred.shape) > 1 else 1
  y_pred = tf.math.divide(tf.reduce_sum(y_pred), divisor)
  return y_pred

# This function gets the accuracy
def accuracy(y_true:tf.Tensor, y_pred:tf.Tensor)->float:
  logging.debug("y_true = " + str(y_true.shape) + \
                " y_pred = " + str(y_pred.shape))
  y_true = tf.transpose(y_true)
  logging.debug("y_true = " + str(y_true.shape) + \
                " y_pred = " + str(y_pred.shape))
  eq = tf.math.equal(y_true, y_pred)
  return float(eq[eq == True].shape[0]) / float(eq.shape[0])

# This function finds the mean absolute error
def mean_absolute_error(y_true:tf.Tensor, y_pred:tf.Tensor)->tf.Variable:
  y_true = tf.transpose(y_true)
  logging.debug(y_true.shape, y_pred.shape)
  abs_dist = tf.math.abs(tf.math.subtract(y_true, y_pred))
  return tf.reduce_mean(abs_dist)

# This function finds the mean square error
def mean_square_error(y_true:tf.Tensor, y_pred:tf.Tensor)->tf.Variable:
  return tf.reduce_mean(tf.math.squared_difference(y_true, y_pred))

# This function finds the mean square error after multiplying each
# value by 255 (since we would have converted the range [0,255] to the range
# [0, 1])
def mean_square_error_image(yt:tf.Tensor, yp:tf.Tensor)->tf.Variable:
  y1 = tf.math.multiply(yt, 255.0)
  y2 = tf.math.multiply(yp, 255.0)
  return mean_square_error(tf.transpose(y1), y2)

# Neural network class for reuse
class NeuralNetwork(object):

  def __init__(self, network, dropout_probability=0.3):
    # the network contains the matrices that describe each layer
    self.network = network
    # The dropout probability
    self.prob_dropout = dropout_probability
    # Store the result of each iteration
    self.iterations = []
    self.train_accuracies = []
    self.test_accuracies = []
    self.train_losses = []
    self.test_losses = []

  def print_drop_matrix_stat(self, d:tf.Tensor)->None:
    ones = tf.ones(shape=d.shape)
    all = tf.reduce_sum(ones)
    this = tf.reduce_sum(d)
    logging.debug(tf.divide(this, all))

  # A slight variation of convention is used here, the dropout is applied to
  # the input of a layer, rather than the output. This doesn't make any
  # difference as the definition of the model is changed to achieve the same
  # result
  def get_dropout_input(self, input:tf.Tensor)->tf.Tensor:
    prob_dropout = self.prob_dropout
    prob_threshold = 1 - prob_dropout
    drop_matrix = tf.random.uniform(shape=input.shape, minval=0,  maxval=1)
    drop_matrix = tf.math.less(drop_matrix, prob_threshold)
    drop_matrix = tf.cast(drop_matrix, dtype=tf.float32)
    input = tf.math.multiply(input, drop_matrix)
    input = tf.math.divide(input, prob_threshold)
    return input

  # The forward pass function, pushes the input through all the layers
  # the parameter layers is a list of tuples, each tuple is of the form
  # (
  #    weight-matrix:list,
  #    bias-matrix:list,
  #    activation_function, 
  #    is_dropout_enabled:bool
  # )
  def forward_pass(self, tr_x, tr_y, layers:list):
    input = tf.transpose(tr_x)
    for weight, bias, activation, dropout in layers:
        # If dropout is enabled on the layer, apply dropout on the input
        if dropout:
          input = self.get_dropout_input(input)
        input = activation(tf.matmul(weight, input) + bias)
    return input
  
  def predict(self, tr_x, tr_y):
    input = tf.transpose(tr_x)
    for weight, bias, activation, dropout in self.network:
      input = activation(tf.matmul(weight, input) + bias)
    return input
  
  """
  Take a network which is a list of tuples, and flatten it
  to a list, so that we can call adam and gradient on it without 
  without any ugly code
  """
  def flatten_network(self, network:list)->list:
    out = []
    for weight, bias, act, dropout in network:
      out.append(weight)
      out.append(bias)
    return out
  
  """
  Use the adam optimizer to optimize a network based on the 
  gradients passed to it
  """
  def optimize_network(self, network:list, gradients:list)->None:
    adam_optimizer = tf.keras.optimizers.Adam()
    learnable_array = []
    adam_optimizer.apply_gradients(zip(gradients, \
                                       self.flatten_network(network)))
  
  def should_log(self, i:int, iterations:int):
    return (i == (iterations - 1)) or all([i == '0' for i in list(str(i)[1:])])
  
  """
  Train the network for a given number of iterations
  """
  def train_network(self, \
                    iterations:int, \
                    tr_x:tf.Tensor, tr_y:tf.Tensor, \
                    te_x:tf.Tensor, te_y:tf.Tensor, \
                    loss_fn, acc_fn, \
                    do_logging=True, \
                    use_argmax=True)->None:
    for i in range(iterations):

      # Call forward pass on the network and use the gradient tape
      with tf.GradientTape() as tape:
        y_pred = self.forward_pass(tr_x, tr_y, self.network)
        loss = loss_fn(tr_y, y_pred)
      gradients = tape.gradient(loss, self.flatten_network(self.network))

      # Record the training accuracy before updating the weights
      if (use_argmax):
        tr_acc = acc = acc_fn(tf.argmax(tr_y), tf.argmax(y_pred)) \
                if None != acc_fn else float("NaN")
      else:
        tr_acc = acc = acc_fn(tr_y, y_pred) if None != acc_fn else float("NaN")
      tr_loss = loss

      # Log the same if required
      if do_logging and self.should_log(i, iterations):
        printstr = f"Iteration {i:5d}  TRAIN LOSS - {loss:5.5f}  "
        printstr = printstr + f"TRAIN ACCURACY - {acc:5.5f}"

      # Now run the prediction on the test set, and record the accuracy
      # We haven't updated the weights yet
      y_pred = self.predict(te_x, te_y)
      te_loss = loss = loss_fn(te_y, y_pred)
      if (use_argmax):
        te_acc = acc = acc_fn(tf.argmax(te_y), tf.argmax(y_pred)) \
                if None != acc_fn else float("NaN")
      else:
        te_acc = acc = acc_fn(te_y, y_pred) if None != acc_fn else float("NaN")

      # Log if required
      if do_logging and self.should_log(i, iterations):
        printstr = printstr + f"  TEST_LOSS = {loss:5.5f}  "
        printstr = printstr + f"TEST ACCURACY = {acc:5.5f}"
        logging.info(printstr)

      # Record the performance of this run
      self.iterations.append(i)
      self.train_accuracies.append(tr_acc)
      self.train_losses.append(tr_loss)
      self.test_accuracies.append(te_acc)
      self.test_losses.append(te_loss)

      # Now use ADAM and update the weights
      self.optimize_network(self.network, gradients)

    self.iterations = np.array(self.iterations)
    self.train_accuracies = np.array(self.train_accuracies)
    self.train_losses = np.array(self.train_losses)
    self.test_accuracies = np.array(self.test_accuracies)
    self.test_losses = np.array(self.test_losses)

# Question 1_1_2

## Create the network

In [5]:
def q_1_1_2():
  tr_x, tr_y, te_x, te_y = get_fashion_mnist_data()
                              
  # Layer 1 has 200 neurons, and 784 features,
  # hence the array should be (200, 784)
  network = [
            # Layer 1, 300 nodes, ReLu
            (make_layer([300, 784], "L1 Weights"),
             make_layer([300, 1],   "L1 Bias"),
             tf.nn.relu,
             False),
            # Layer 2, 100 nodes, Relu
            (make_layer([100, 300],  "L2 Weights"),
             make_layer([100, 1],    "L2 Bias"),
             tf.nn.relu,
             False),
            # Layer 2, 10 nodes, Softmax
            (make_layer([10, 100],  "L2 Weights"),
             make_layer([10, 1],    "L2 Bias"),
             softmax,
             False),
  ]

  nn = NeuralNetwork(network)
  nn.train_network(1000, tr_x, tr_y, te_x, te_y, cross_entropy, accuracy)
  return nn

## Run the network and produce the graphs

4423680/4422102 [==============================] - 0s 0us/step
2021-04-17 09:40:47,795 - root - INFO - Iteration     0  TRAIN LOSS - 2.30580  TRAIN ACCURACY - 0.10192  TEST_LOSS = 2.30671  TEST ACCURACY = 0.10450
2021-04-17 09:40:47,868 - root - INFO - Iteration     1  TRAIN LOSS - 2.18030  TRAIN ACCURACY - 0.20477  TEST_LOSS = 2.18138  TEST ACCURACY = 0.20180
2021-04-17 09:40:47,911 - root - INFO - Iteration     2  TRAIN LOSS - 2.05935  TRAIN ACCURACY - 0.43343  TEST_LOSS = 2.06091  TEST ACCURACY = 0.42710
2021-04-17 09:40:47,952 - root - INFO - Iteration     3  TRAIN LOSS - 1.93008  TRAIN ACCURACY - 0.55047  TEST_LOSS = 1.93275  TEST ACCURACY = 0.54010
2021-04-17 09:40:47,995 - root - INFO - Iteration     4  TRAIN LOSS - 1.79285  TRAIN ACCURACY - 0.56933  TEST_LOSS = 1.79698  TEST ACCURACY = 0.56310
2021-04-17 09:40:48,038 - root - INFO - Iteration     5  TRAIN LOSS - 1.65413  TRAIN ACCURACY - 0.59420  TEST_LOSS = 1.65879  TEST ACCURACY = 0.58700
2021-04-17 09:40:48,080 - root - INFO

In [6]:
import matplotlib.pyplot as plt

nn = q_1_1_2()

plt.rcParams["figure.figsize"] = (20, 5)

def norm(x):
  minimum = np.min(x)
  maximum = np.max(x)
  x = (x - minimum) / (maximum - minimum)
  return x

def norm(x):
  return x

fig, ax = plt.subplots(1, 2)

ax[0].plot(nn.iterations, norm(nn.test_losses), label="test losses")
ax[0].plot(nn.iterations, norm(nn.train_losses), label="train losses")
ax[0].set_title("loss")
ax[0].legend()
ax[1].plot(nn.iterations, norm(nn.test_accuracies), label="test accuracies")
ax[1].plot(nn.iterations, norm(nn.train_accuracies), label="train accuracies")
ax[1].set_title("accuracy")
ax[1].legend()
plt.show()

## Now vary the number of neurons to see if it makes a difference

In [7]:
import matplotlib.pyplot as plt

def q_1_1_2_vary_neurons(scale_factor:float):
  tr_x, tr_y, te_x, te_y = get_fashion_mnist_data()
                              
  # Layer 1 has 200 neurons, and 784 features,
  # hence the array should be (200, 784)
  network = [
            # Layer 1, 300 nodes, ReLu
            (make_layer([int(300 * scale_factor), 784], "L1 Weights"),
             make_layer([int(300 * scale_factor), 1],   "L1 Bias"),
             tf.nn.relu,
             False),
            # Layer 2, 100 nodes, Relu
            (make_layer([int(100 * scale_factor), int(300 * scale_factor)],  "L2 Weights"),
             make_layer([int(100 * scale_factor), 1],    "L2 Bias"),
             tf.nn.relu,
             False),
            (make_layer([10, int(100 * scale_factor)],  "L2 Weights"),
             make_layer([10, 1],    "L2 Bias"),
             softmax,
             False),
  ]

  nn = NeuralNetwork(network)
  nn.train_network(1000, tr_x, tr_y, te_x, te_y, cross_entropy, accuracy, \
                   do_logging=False)
  return int(300 * scale_factor), \
          max(nn.train_accuracies), min(nn.train_losses), \
          max(nn.test_accuracies), min(nn.test_losses), \
          nn


n_neurons = list()
train_losses = list()
test_losses = list()
train_accuracies = list()
test_accuracies = list()
nns = list()

for i in (0.5, 1, 2, 4):
  neurons, train_accuracy, train_loss, test_accuracy, test_loss, nn \
    = q_1_1_2_vary_neurons(i)
  n_neurons.append(neurons)
  train_losses.append(train_loss)
  test_losses.append(test_loss)
  train_accuracies.append(train_accuracy)
  test_accuracies.append(test_accuracy)
  nns.append(nn)

plt.rcParams["figure.figsize"] = (30, 45)

def EWMA(x, alpha):
  s = x[0]
  arr = [s]
  for i in x[1:]:
    s = alpha * i + (1 - alpha) * s
    arr.append(s)
  return arr

def norm(x):
  return EWMA(x, 0.01)

fig, ax = plt.subplots(3, 2)

color_arr = ['b', 'r', 'g', 'c', 'm', 'y', 'k']

ax[0][0].plot(n_neurons, test_losses, label="test losses")
ax[0][0].plot(n_neurons, train_losses, label="train losses")
ax[0][0].set_title("loss")
ax[0][0].legend()
ax[0][1].plot(n_neurons, test_accuracies, label="test accuracies")
ax[0][1].plot(n_neurons, train_accuracies, label="train accuracies")
ax[0][1].set_title("accuracy")
ax[0][1].legend()

for i, neurons, nn in zip(list(range(len(n_neurons))), n_neurons, nns):
  ax[2][0].plot(nn.iterations, norm(nn.test_losses), label=f"EWMA test losses neurons={neurons}", linestyle='dotted', color=color_arr[i])
  ax[2][0].plot(nn.iterations, norm(nn.train_losses), label=f"EWMA train losses neurons={neurons}", linestyle='solid', color=color_arr[i])
  ax[2][0].set_title("loss")
  ax[2][1].plot(nn.iterations, norm(nn.test_accuracies), label=f"EWMA test accuracies neurons={neurons}", linestyle='dotted', color=color_arr[i])
  ax[2][1].plot(nn.iterations, norm(nn.train_accuracies), label=f"EWMA train accuracies neurons={neurons}", linestyle='solid', color=color_arr[i])
  ax[2][1].set_title("accuracy")
ax[2][0].legend()
ax[2][1].legend()

def norm(x):
  return x

for i, neurons, nn in zip(list(range(len(n_neurons))), n_neurons, nns):
  ax[1][0].plot(nn.iterations, nn.test_losses, label=f"test losses neurons={neurons}", linestyle='dotted', color=color_arr[i])
  ax[1][0].plot(nn.iterations, nn.train_losses, label=f"train losses neurons={neurons}", linestyle='solid', color=color_arr[i])
  ax[1][0].set_title("loss")
  ax[1][1].plot(nn.iterations, nn.test_accuracies, label=f"test accuracies neurons={neurons}", linestyle='dotted', color=color_arr[i])
  ax[1][1].plot(nn.iterations, nn.train_accuracies, label=f"train accuracies neurons={neurons}", linestyle='solid', color=color_arr[i])
  ax[1][1].set_title("accuracy")
ax[1][0].legend()
ax[1][1].legend()


plt.show()